In [ ]:
!pip install geopandas==0.11.0
!pip install pandas==1.4.3
!pip install numpy==1.22.4
!pip install folium matplotlib mapclassify

import zipfile
import os
from glob import glob
path = os.getcwd()
import pandas as pd
import numpy as np
# path = os.getcwd()
path = '/content/drive/My Drive/python_geomap/'
os.getcwd()
import matplotlib
import geopandas as gpd # GeoPandas(지오판다스)

import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from pyproj import Proj, transform

import pickle
# pickle.dump(hyd_df_merged, open(os.path.join(path,"data","merged","merged_1","test.p",), "wb"))     
# test = pickle.load(open(os.path.join(path,"data","merged","merged_1","test.p",), "rb"))  

from copy import deepcopy
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 
pd.set_option("display.precision", 10)

from matplotlib.pyplot import legend
import plotly.express as px
import plotly.graph_objects as go


from google.colab import drive
drive.mount('/content/drive')

if pd.__version__ != '1.4.3':
  raise Exception('pd version CHECK!!')

* pec 지도그리기 파트

In [3]:
def take_map_df():
    class Rich():
        def __init__ (self, rich_did, lu_rch_did, ru_rch_did, stream_flux, c_area, geometry):
            self.rich_did = rich_did
            self.lu_rch_did = lu_rch_did
            self.ru_rch_did = ru_rch_did
            self.stream_flux = stream_flux               ## 단위 m3/s
            self.c_area = c_area
            self.geometry = geometry
            self.stream_flux_estimated = None
            self.lu_child = None
            self.ru_child = None
            self.stp_family = None
            self.stp_site = False

            self.parent = None

            self.velocity = None                         ## 단위 m/s
            self.velocity_estimated = None
            self.rch_len = None
            self.r_time = None                           ## 단위 hour

            self.PEC_est = None
            self.mass_g = None
        
        def take_lu_child(self,lu_rich_class,):
            self.lu_child = lu_rich_class
            
        def take_ru_child(self,ru_rich_class,):
            self.ru_child = ru_rich_class

        def estimate_flux(self, parents_true_flux, parents_true_carea):
            # ~np.isnan(self.stream_flux_estimated)
            if self.stream_flux is None:
                self.stream_flux_estimated = parents_true_flux *(self.c_area/parents_true_carea)

            elif self.stream_flux is not None:
                print('참값 있음')

    line_gdf_rtime = pickle.load(open(os.path.join(path,"data","lind_gdf_rtime.p",), "rb"))
    line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '2004170700', 'MB_NM'] = '내성천'
    stp_gdf_info = pickle.load(open(os.path.join(path,"data","stp_gdf_pop.p",), "rb"))
    stp_gdf_info.loc[:,'방류량'] = np.float64(stp_gdf_info.loc[:,'방류량'])

    data_rich = line_gdf_rtime.loc[:,['RCH_DID', 'LU_RCH_DID', 'RU_RCH_DID','CUM_AREA', 'geometry_x']].values
    rich_class_list = []
    for i in range(len(line_gdf_rtime)):
        i_info_rich = data_rich[i]
        rich_class_list.append(Rich(i_info_rich[0],i_info_rich[1],i_info_rich[2],None ,i_info_rich[3],i_info_rich[4]))

    line_gdf_rtime.loc[:,'rich_class'] = rich_class_list

    # 유량	est_flux
    line_gdf_rtime.loc[:,'est_flux'] = line_gdf_rtime.loc[:,'est_flux'] * 86400
    line_gdf_rtime.loc[:,'유량'] = line_gdf_rtime.loc[:,'유량'] * 86400
    for i in range(len(line_gdf_rtime)):
        try:
            line_gdf_rtime.loc[i,'rich_class'].stream_flux_estimated = line_gdf_rtime.loc[i,'est_flux'] 
        except:
            pass
        try:
            line_gdf_rtime.loc[i,'rich_class'].stream_flux = line_gdf_rtime.loc[i,'유량'] 
        except:
            pass

        try:
            line_gdf_rtime.loc[i,'rich_class'].r_time = line_gdf_rtime.loc[i,'r_time']
        except:
            pass
        try:
            line_gdf_rtime.loc[i,'rich_class'].parent = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == line_gdf_rtime.loc[i,'parent_rch_did'],'rich_class'].values[0]
        except:
            pass

    k = 0.00193 # 반응속도상수 반감기 역수 h-1

    koc = 484800 # 사용자 입력값, L/kg
    Foc_susp = 0.1 # kg_oc*kg_solid-1
    SUSP_water = 4 # mg_solid*L_water-1
    constant_term = koc * Foc_susp * SUSP_water * (10**-6)   # 무차원 상수


    removal_ratio = 0.641 # 물질별로 계산해서 나오는 값 제거율
    base_c = 0.0000044 # 물질별로 나오는 값인것으로 알고있음, 배경농도 g/m3
    e_p = 0.000085 #인당 물질 배출량,  g/(d*명)

    # mass_g

    def cal_mass_down(stp_r, mass_g_out, k_constant):
        if stp_r.parent is not None:
            down_mass = mass_g_out * np.exp(-1*k_constant*stp_r.r_time) #구간 잔존률 * 위에서온 mass

            if stp_r.parent.mass_g is None:
                stp_r.parent.mass_g = down_mass  
            else:
                before_mass = stp_r.parent.mass_g
                stp_r.parent.mass_g = before_mass + down_mass

            cal_mass_down(stp_r.parent, down_mass, k_constant)
        

    # 우선 농도계산은 하지 않고 강줄기별 mass만 계산
    for i in range(len(stp_gdf_info)):
        i_info = stp_gdf_info.loc[i,['Effluent', '인구수', 'RCH_DID']]
        
        population = i_info.인구수
        stp_r = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == i_info.RCH_DID].rich_class.values[0]
        if stp_r.r_time is not None:
        

            mass_g = (e_p * population)  # g/day   _ 물질 배출량
            mass_g_out = mass_g*(1-removal_ratio)/(1 + constant_term)   # 처리효과 흡착??효과 고려한 물질 out 양


            # 총 물질량 합산 위에서 흘러온것 + stp에서 배출한 것
            try:
                mass_before = stp_r.mass_g
                stp_r.mass_g = mass_before + mass_g_out  # 앞단에서 모든 강줄기에 배경농도는 미리 더해놓음
            except:
                stp_r.mass_g = mass_g_out

            cal_mass_down(stp_r, mass_g_out, k)


    # 0.2보다 크면 방류량 추가 요청
    def parent_flux_est2(i_r,eff):
        
        if (i_r.parent is not None) and (i_r.parent.stream_flux_estimated is not None):
            before_stream = i_r.parent.stream_flux_estimated
            i_r.parent.stream_flux_estimated = before_stream + eff
            
            # if i_r.parent.parent is not None:
            parent_flux_est2(i_r.parent,eff)


    for i in range(len(stp_gdf_info)):
        eff_stp = stp_gdf_info.loc[i,'Effluent']
        rch_did = stp_gdf_info.loc[i,'RCH_DID']

        matched_line = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == rch_did]
        i_rich = matched_line.rich_class.values[0]
        if i_rich.stream_flux_estimated is not None:
            
            flux = i_rich.stream_flux_estimated
            if eff_stp / flux >= 0.2:
                
                i_rich.stream_flux_estimated = flux + eff_stp
                parent_flux_est2(i_rich,eff_stp)
            else:            
                i_rich = i_rich
                i_rich.stream_flux_estimated = flux



        elif i_rich.stream_flux is not None:
            
            flux = i_rich.stream_flux
            if eff_stp / flux >= 0.2:
                
                i_rich.stream_flux_estimated = flux + eff_stp
                parent_flux_est2(i_rich,eff_stp)
            else:            
                i_rich = i_rich
                i_rich.stream_flux_estimated = flux
        
    for i in range(len(line_gdf_rtime)):
        i_rich = line_gdf_rtime.loc[i,'rich_class']
        
        if np.isnan(i_rich.stream_flux_estimated):
            if ~np.isnan(line_gdf_rtime.loc[i,'유량']):
                i_rich.stream_flux_estimated = line_gdf_rtime.loc[i,'유량']

        

    for i in range(len(line_gdf_rtime)):
        line_gdf_rtime.loc[i,'est_flux'] = line_gdf_rtime.loc[i,'rich_class'].stream_flux_estimated

    # 합천창녕보 리치 중 한곳 바로 위 리치로 유량 교체 요청
    temp_est_flux = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '2014060100','est_flux'].values[0]
    line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '2014040400','est_flux'] = temp_est_flux
    line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '2014040400','rich_class'].values[0].stream_flux_estimated = temp_est_flux

    # 갑천 유량측정지 본류합류전까지 유량 대체값 조정 요청
    parent_est_flux = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '3009061401','est_flux'].values[0]
    parent = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '3009061401','rich_class'].values[0].parent
    while True:
        parent.stream_flux_estimated = parent_est_flux
        line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == parent.rich_did,'est_flux'] = parent_est_flux
        if parent.rich_did == '3010010100':
            break
        parent = parent.parent


    # 추정 유량을 통해 농도 환산
    for i in range(len(line_gdf_rtime)):
        i_r = line_gdf_rtime.loc[i,'rich_class']
        if i_r.mass_g is not None:
            
            pec_est = (i_r.mass_g / (line_gdf_rtime.loc[i,'est_flux'])) + base_c

            line_gdf_rtime.loc[i,'rich_class'].PEC_est = pec_est
            line_gdf_rtime.loc[i,'PEC_WS'] = pec_est

    # # 안동댐 수문 하류 농도 배출농도로 일괄처리 요청
    # temp_rch = line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == '2001110900','rich_class'].values[0]
    
    # parent = temp_rch.parent
    # while True:
    #     if parent.rich_did == '2003040300':
    #         break
    #     parent.PEC_est = temp_rch.PEC_est
    #     line_gdf_rtime.loc[line_gdf_rtime.RCH_DID == parent.rich_did, 'PEC_WS'] = temp_rch.PEC_est
    #     parent = parent.parent


        

    line_gdf_rtime.loc[:,'mass'] = [r.mass_g for r in line_gdf_rtime.rich_class.values]
    line_gdf_last = gpd.GeoDataFrame(line_gdf_rtime.loc[~line_gdf_rtime.r_time.isna(),['RCH_DID','MB_NM','RCH_LEN','geometry_x','PEC_WS','r_time','유량','est_flux']], geometry = 'geometry_x')
    line_gdf_last.columns = ['RCH_DID','MB_NM','RCH_LEN','geometry_x','PEC_WS','구간체류시간(h)','유량(m3/day)','추정유량(m3/day)']
    line_gdf_last = line_gdf_last.loc[:,['geometry_x','RCH_DID','MB_NM','RCH_LEN','유량(m3/day)','추정유량(m3/day)','구간체류시간(h)','PEC_WS',]]
    line_gdf_last.index = range(len(line_gdf_last))





    stp_gdf_info_2 = stp_gdf_info.loc[[b in line_gdf_last.RCH_DID.tolist() for b in stp_gdf_info.RCH_DID],['RCH_DID','시설명', '방류량', 'geometry', '인구수']]

    test = pd.merge(stp_gdf_info_2, line_gdf_last, how = 'left', on = 'RCH_DID')
    for i in range(len(test)):
        
        if ~ np.isnan(test.loc[i,'추정유량(m3/day)']):
            d_factor_i = (test.loc[i,'추정유량(m3/day)']) / (test.loc[i,'방류량'])
            if 0< d_factor_i < 1:
                # 희석배율 1미만인지역 1로 맞추고 변경된 추정유량으로 PEC_WS 변경
                test.loc[i,'희석배율'] = 1
                before_est_flux = test.loc[i,'추정유량(m3/day)']
                grad_flux = before_est_flux/test.loc[i,'방류량']
                test.loc[i,'추정유량(m3/day)'] = test.loc[i,'방류량']

                line_gdf_last.loc[line_gdf_last.RCH_DID == test.loc[i,'RCH_DID'],'추정유량(m3/day)'] = test.loc[i,'방류량']

                # line_gdf_rtime.loc[i,'PEC_WS']
                line_gdf_last.loc[line_gdf_last.RCH_DID == test.loc[i,'RCH_DID'],'PEC_WS'] = (line_gdf_last.loc[line_gdf_last.RCH_DID == test.loc[i,'RCH_DID'],'PEC_WS'] - base_c) * grad_flux + base_c
                test.loc[i,'PEC_WS'] = line_gdf_last.loc[line_gdf_last.RCH_DID == test.loc[i,'RCH_DID'],'PEC_WS'].values[0]

            else:
                test.loc[i,'희석배율'] = d_factor_i

            PEC_local_i = (test.loc[i,'인구수'] * e_p * (1-removal_ratio)/(1 + constant_term)) / (test.loc[i,'추정유량(m3/day)'])


        test.loc[i,'C_local'] = (test.loc[i,'인구수'] * e_p * (1-removal_ratio)/(1 + constant_term)) / (test.loc[i,'방류량'])
        test.loc[i,'PEC_local'] = PEC_local_i
        


    remove_ind_list = []
    for i in range(len(line_gdf_last)):
        if line_gdf_last.loc[i,'RCH_DID'] not in set(test.RCH_DID):
            remove_ind_list.append(i)

    line_gdf_last_remove_dup = line_gdf_last.loc[remove_ind_list]
    line_gdf_last_remove_dup.index = range(len(line_gdf_last_remove_dup))

    total_df = pd.concat([test,line_gdf_last_remove_dup])
    total_df.index = range(len(total_df))

    test = test.loc[:,['RCH_DID','시설명','방류량','geometry', '인구수','희석배율','C_local','PEC_local',]]

    bpr_pec = ((10000 * e_p * (1-removal_ratio)/(1 + constant_term)) / 20000) + base_c

    line_gdf_last = gpd.GeoDataFrame(pd.merge(line_gdf_last, line_gdf_rtime.loc[:,['RCH_DID', 'rich_class']], how = 'left', on='RCH_DID'), geometry = 'geometry_x')
    return line_gdf_last, test, total_df, bpr_pec,


In [4]:
# 그래프그릴 데이터 t_df
line_gdf, stp_point_gdf, t_df, b_pec = take_map_df()

stp_point_gdf2 = gpd.GeoDataFrame(pd.merge(line_gdf.loc[:,['RCH_DID', 'MB_NM']], stp_point_gdf, how = 'right', on='RCH_DID'),geometry = 'geometry')

t_col_list = ['MB_NM',
'시설명',
'방류량',
'인구수',
'RCH_LEN',
'구간체류시간(h)',
'희석배율',
'C_local',
'PEC_local',
'PEC_WS',]

t_df = t_df.loc[:,t_col_list]

def get_geo_map(MB: str, stp_geoinfo_gdf: gpd.GeoDataFrame, line_geoinfo_gdf: gpd.GeoDataFrame):
    
    if MB == '전국':
        pass
    else:
        stp_geoinfo_gdf = stp_geoinfo_gdf.loc[stp_geoinfo_gdf.MB_NM ==MB]
        line_geoinfo_gdf = line_geoinfo_gdf.loc[line_geoinfo_gdf.MB_NM ==MB]
    ## 지도그리는 함수부분 ##
    import folium
    color_list_1 = stp_geoinfo_gdf.PEC_local.values.tolist() + line_geoinfo_gdf.PEC_WS.tolist()
    color_list_1.sort()

    # index_list = [0.000002,0.000004,0.0000045,0.00000468,0.00000481,0.00000513,0.00000569,0.00000704,0.00000809,0.0000111,0.0000196,]
    index_list = [0.000001,0.000002,0.0000021,0.0000022,0.0000023,0.0000024]
    col_bar_list = []


    # 1지망
    # r_start = 170
    # g_start = 200
    # b_start = 230

    # r_end = 241
    # g_end = 18
    # b_end = 152

    r_start = 170
    g_start = 200
    b_start = 230

    r_end = 241
    g_end = 0
    b_end = 102




    for i in range(len(index_list)):

        r = r_start - (r_start - r_end)*(i/len(index_list))
        g = g_start - (g_start - g_end)*(i/len(index_list))
        b = b_start - (b_start - b_end)*(i/len(index_list))
    
        col_bar_list.append((r,g,b))

    import branca.colormap as cmp

    con_step = cmp.LinearColormap(
        colors=col_bar_list,
        # index=index_list,
        vmin=np.min(color_list_1),
        vmax=np.max(color_list_1),
        caption='PEC(mg/L)'
    )

    import branca.colormap as cmp
    import folium

    m = line_geoinfo_gdf.explore(tiles = "CartoDB positron", column = 'PEC_WS', legend = True, cmap = con_step,)
    stp_geoinfo_gdf.explore(m=m, column = 'PEC_local', cmap = con_step, marker_type = folium.Circle(radius=300, fill = 'black'),)
    return m, stp_geoinfo_gdf, line_geoinfo_gdf

map_1, stp_info_filt2, line_geo_info2 = get_geo_map('전국',stp_point_gdf2, line_gdf.iloc[:,:-1])
map_1

AttributeError: ignored

In [ ]:
# 안성천지도만 뽑기기
map_1, stp_info_filt2, line_geo_info2 = get_geo_map('안성천',stp_point_gdf2, line_gdf.iloc[:,:-1])
map_1

* pec 하류로부터 연결관계 지도 그리기 파트

In [ ]:
def get_stream_pec_plot(site: str,):

  line_gdf_local, stp_point_gdf_local, _, _ = take_map_df()
  stp_point_gdf2 = gpd.GeoDataFrame(pd.merge(line_gdf.loc[:,['RCH_DID', 'MB_NM']], stp_point_gdf, how = 'right', on='RCH_DID'),geometry = 'geometry')

  xx = site
  rch_list = []
  rch_len_list = []
  stp_list = []

  if xx != '전국':
      mb_gdf = line_gdf_local.loc[line_gdf_local.MB_NM == xx]
      print(mb_gdf.columns.tolist())
  else:
      print('전국 ㄴㄴ')
      return None

  for i in mb_gdf.index.to_list():
      ru_did = mb_gdf.rich_class[i].ru_rch_did
      lu_did = mb_gdf.rich_class[i].lu_rch_did
      
      #ru_rch_did, lu_rch_did, PEC_est
      

      if ru_did is None or ru_did not in mb_gdf.RCH_DID.tolist():
          if lu_did is None or lu_did not in mb_gdf.RCH_DID.tolist():
              rch_list_i = []
              rch_list_i.append(mb_gdf.rich_class[i].PEC_est)

              rch_len_list_i = []
              rch_len_list_i.append(line_gdf_local.loc[line_gdf_local.RCH_DID == mb_gdf.rich_class[i].rich_did].RCH_LEN.values[0])

              stp_list_i = []
              if len(stp_point_gdf_local.loc[stp_point_gdf_local.RCH_DID == mb_gdf.rich_class[i].rich_did, '시설명']) == 1:
                  stp_list_i.append(stp_point_gdf_local.loc[stp_point_gdf_local.RCH_DID == mb_gdf.rich_class[i].rich_did, '시설명'].values[0])
              else:
                  stp_list_i.append(None)

              ######## 이 뒤로 stp_list_i 붙이는 것 수정
              if mb_gdf.rich_class[i].parent is not None:
                  parent = mb_gdf.rich_class[i].parent
                  while True:
                      if parent.rich_did in mb_gdf.RCH_DID.tolist():
                          
                          rch_list_i.append(parent.PEC_est)
                          rch_len_list_i.append(line_gdf_local.loc[line_gdf_local.RCH_DID == parent.rich_did].RCH_LEN.values[0])

                          if len(stp_point_gdf_local.loc[stp_point_gdf_local.RCH_DID == parent.rich_did, '시설명']) == 1:
                              stp_list_i.append(stp_point_gdf_local.loc[stp_point_gdf_local.RCH_DID == parent.rich_did, '시설명'].values[0])
                          else:
                              stp_list_i.append(None)


                          if parent.parent is not None:
                              parent = parent.parent
                          else:
                              rch_len_list.append(rch_len_list_i)
                              rch_list.append(rch_list_i)
                              stp_list.append(stp_list_i)
                              break
                      else:
                          rch_len_list.append(rch_len_list_i)
                          rch_list.append(rch_list_i)
                          stp_list.append(stp_list_i)
                          break
                  

  for i in range(len(rch_len_list)):
      tem_list = deepcopy(rch_len_list[i])
      tem_list.reverse()
      tem_list = np.cumsum(tem_list).tolist()
      tem_list.reverse()
      rch_len_list[i] = tem_list

  df_list = []
  for i in range(len(rch_len_list)):
      t_df = pd.DataFrame(np.reshape(rch_list[i],(-1,1)), index=rch_len_list[i])
      df_list.append(t_df)

  df_list_stp = []
  for i in range(len(stp_list)):
      t_df = pd.DataFrame(np.reshape(stp_list[i],(-1,1)), index=rch_len_list[i])
      df_list_stp.append(t_df)
  stp_df = pd.concat(df_list_stp, axis=1).sort_index()
  stp_info_list_by_clen = []
  for i in range( len(stp_df)):
      stp_info_list_by_clen_i = []
      for i in list(set(stp_df.iloc[i].values)):
          if type(i) == str:
              stp_info_list_by_clen_i.append(i)
      if len(stp_info_list_by_clen_i) == 0:
          stp_info_list_by_clen_i.append(None)
      stp_info_list_by_clen.extend(stp_info_list_by_clen_i)
              
  stp_df.loc[:,'stp_name'] = stp_info_list_by_clen
  stp_df.loc[:,'len'] = stp_df.index.tolist()
  stp_df.index = range(len(stp_df))
  stp_df = stp_df.loc[:,['stp_name', 'len']]


  df_list_2 = []
  for i in range(len(df_list)):
      t_df = deepcopy(df_list[i])
      t_df.loc[:,'len'] = t_df.index.tolist()
      t_df.rename(columns = {0:'PEC_ws',},inplace=True)
      t_df.index = range(len(t_df))
      t_df.loc[:,'rch'] = i
      df_list_2.append(t_df)

  t_df3 = pd.concat(df_list_2, axis=0)

  df_by_lenindex = pd.DataFrame(list(set(t_df3.len)), columns=['len']) # 듬성듬성하지만 빠름
  for i in range(len(df_list_2)):
      t_df = deepcopy(df_list_2[i])
      filled_df = pd.merge(df_by_lenindex, t_df, how='left', on = 'len').sort_values('len').fillna(method='backfill')
      df_list_2[i] = filled_df.dropna()

  filled_pec_line_df = pd.concat(df_list_2, axis=0)
  stp_df = pd.merge(stp_df.dropna(),t_df3.loc[:,['PEC_ws', 'len']], how='left', on = 'len').drop_duplicates()
  stp_df.index = range(len(stp_df))

  color_discrete_map = {}
  for i in list(set(filled_pec_line_df.rch)):
    color_discrete_map[i] = 'rgb(0,0,150)'



  fig = px.line(filled_pec_line_df, 
                x='len', # x축
                y='PEC_ws',  # y축
                color='rch',
                title=f'PEC_ws({xx})', # Title 
                color_discrete_map=color_discrete_map,
                )
  # fig.update_layout(showlegend=False)

  color_list = [
          'rgba(255, 51, 51, .9)',
          'rgba(255, 122, 51, .9)',
          'rgba(255, 221, 51, .9)',
          'rgba(168, 255, 51, .9)',
          'rgba(51, 255, 209, .9)',
          'rgba(51, 110, 255, .9)',
          'rgba(212, 51, 255, .9)',
          'rgba(255, 51, 131, .9)',
          'rgba(183, 142, 158, .9)',
          'rgba(131, 45, 79, .9)',
          'rgba(49,   182, 100, .9)',
          'rgba(180, 80, 40, .9)',
          'rgba(30, 123, 97, .9)',
          'rgba(190, 51, 233, .9)',
          'rgba(240, 35, 211, .9)',
          'rgba(100, 87, 193, .9)',
          'rgba(30, 179, 142, .9)',
          'rgba(130, 98, 111, .9)',
          'rgba(170, 12, 193, .9)',
  ]

  for i in range(len(stp_df)):
      i_color = color_list[i]
      stp_point = go.Scatter(x=stp_df.loc[i,['len']],
                                  y=stp_df.loc[i,['PEC_ws']],
                                  mode="markers",
                                  # line=go.scatter.Line(color="gray"),
                                  marker={'size':10},
                                  marker_color=i_color,
                                  legendgroup="STP point",
                                  legendgrouptitle_text="STP point",
                                  name=stp_df.loc[i,['stp_name']].values[0],
                                  showlegend=True
                                  )
      # print(stp_df.loc[i,['stp_name']].values[0])

      fig.add_trace(stp_point)
  return fig, filled_pec_line_df, stp_df, line_gdf_local

In [ ]:
xx = '안성천'
stream_chart, len_df, stp_df, line_df = get_stream_pec_plot(xx)
stream_chart.show()

['geometry_x', 'RCH_DID', 'MB_NM', 'RCH_LEN', '유량(m3/day)', '추정유량(m3/day)', '구간체류시간(h)', 'PEC_WS', 'rich_class']


* 데이터 저장 및 원하는 값 뽑기 예시

In [ ]:
# 하철연결그래프 그릴떄 사용되는 데이터(원본에서 하류부터 길이별로 정리됨)
# len: 최하류로부터의 유로연장
# rch: 지류별 넘버(임의지정)
len_df

,len,PEC_ws,rch
1,0.670214,0.0000321708,0.0
0,0.825076,0.0000322342,0.0
3,4.060658,0.0000328733,0.0
2,4.361042,0.0000329487,0.0
4,7.759132,0.0000334577,0.0
...,...,...,...
53,65.123263,0.0000185893,9.0
55,65.749719,0.0000185893,9.0
54,66.210429,0.0000185893,9.0
56,66.556492,0.0000185893,9.0


In [ ]:
#geometry_x 라인데이터
#RCH_DID 소하천 리치 아이디
#MB_NM 중권역명
#유량(m3/day)
#추정유량(m3/day) 유량 추정값
#구간체류시간 구간체류시간 추정값
#PEC_WS 계산된 PEC_WS값
line_df

,geometry_x,RCH_DID,MB_NM,RCH_LEN,유량(m3/day),추정유량(m3/day),구간체류시간(h),PEC_WS,rich_class
0,"LINESTRING (127.32991 37.65439, 127.33001 37.6...",1015080300,청평댐,6.245353,NaN,1.6781496989e+05,2.4621567787,0.0000069832,<__main__.take_map_df.<locals>.Rich object at ...
1,"LINESTRING (127.31141 37.63402, 127.31184 37.6...",1015080901,청평댐,4.536235,NaN,5.1581882857e+05,1.7883571601,0.0000098321,<__main__.take_map_df.<locals>.Rich object at ...
2,"LINESTRING (127.54297 37.71957, 127.54281 37.7...",1014141100,홍천강,3.500069,NaN,2.3988906437e+07,1.3798609325,0.0000044380,<__main__.take_map_df.<locals>.Rich object at ...
3,"LINESTRING (127.34195 37.59370, 127.34185 37.5...",1015081600,청평댐,1.452305,NaN,1.2884197584e+08,0.5725541215,0.0000044820,<__main__.take_map_df.<locals>.Rich object at ...
4,"LINESTRING (127.63290 37.81721, 127.63281 37.8...",1013040400,의암댐,7.817509,NaN,8.7440563993e+07,3.0819607437,0.0000044772,<__main__.take_map_df.<locals>.Rich object at ...
...,...,...,...,...,...,...,...,...,...
915,"LINESTRING (127.00784 37.18721, 127.00768 37.1...",1101130502,안성천,2.319317,NaN,7.0965867940e+05,3.7708557839,0.0000600490,<__main__.take_map_df.<locals>.Rich object at ...
916,"LINESTRING (126.94587 37.32253, 126.94555 37.3...",1101120101,안성천,2.673544,NaN,2.5714240823e+04,4.3467748720,0.0000326878,<__main__.take_map_df.<locals>.Rich object at ...
917,"LINESTRING (126.63534 34.97887, 126.63496 34.9...",5004050602,죽산보,0.667636,NaN,7.8738521553e+06,1.0034406506,0.0000090056,<__main__.take_map_df.<locals>.Rich object at ...
918,"LINESTRING (128.05289 37.09727, 128.05286 37.0...",1003191201,충주댐,8.542114,667345.8162162162,6.6734581622e+05,7.9427577905,0.0000086051,<__main__.take_map_df.<locals>.Rich object at ...


In [ ]:
# 해당하는 중권역 STP 지점들들
stp_df

,stp_name,len,PEC_ws
0,현덕,10.881242,0.0000315881
1,팽성,19.079139,0.0000202792
2,통복,25.583670,0.0000255296
3,장당,30.999587,0.0000494817
4,성환,32.356072,0.0000527633
5,안성,42.453672,0.0000196254
6,정남,46.085510,0.0000575336
7,오산,47.941516,0.0000587793
8,동탄2 공공하수처리시설,51.640354,0.0000451376
9,"수원(제1,제2)",52.369134,0.0000629893


* 데이터 저장
* stp_df라는 데이터를 저장하는 예시

In [ ]:
# 데이터 저장하는 방법 예시
stp_df.to_csv(os.path.join(path,'test.csv'), encoding = 'euc-kr')

* 원하는 컬럼만 얻기
* line_df 라는 데이터 프레임에서 'RCH_DID','RCH_LEN' 컬럼만 얻기 예시

In [ ]:
# 원하는 컬럼만 뽑기 예시
line_df.loc[:, ['RCH_DID','RCH_LEN']]

,RCH_DID,RCH_LEN
0,1015080300,6.245353
1,1015080901,4.536235
2,1014141100,3.500069
3,1015081600,1.452305
4,1013040400,7.817509
...,...,...
915,1101130502,2.319317
916,1101120101,2.673544
917,5004050602,0.667636
918,1003191201,8.542114


* 조건을 만족하는 행과 컬럼 얻기
* 중권역이 안성천인 RCH_DID, RCH_LEN 컬럼만 얻기 예시시

In [ ]:
# 조건을 만족하는 행만 뽑기 예시
line_df.loc[line_df.MB_NM == '안성천', ['RCH_DID','RCH_LEN']]

,RCH_DID,RCH_LEN
683,1101120300,9.616694
684,1101150600,5.025561
685,1101170800,6.794545
686,1101120700,11.585074
687,1101130100,1.022546
688,1101130300,1.623516
689,1101130501,2.340791
690,1101130900,2.638714
691,1101141800,0.124645
692,1101141900,3.720388


In [ ]:
# 모든 컬럼명 뽑기
line_df.columns

Index(['geometry_x', 'RCH_DID', 'MB_NM', 'RCH_LEN', '유량(m3/day)',
       '추정유량(m3/day)', '구간체류시간(h)', 'PEC_WS', 'rich_class'],
      dtype='object')